In [1]:
import batoid
import galsim
import yaml
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%autoreload 2

In [2]:
filename = os.path.join(batoid.datadir, 'HSC', 'HSC.yaml')
config = yaml.load(open(filename))
fiducial_telescope = batoid.parse.parse_optic(config['opticalSystem'])

In [43]:
dkdu = batoid.psf.dkdu(fiducial_telescope, 0.0, 0.01, 750e-9, 1024)
print(dkdu)

[[ -4.49226436e+05   6.62373549e-02]
 [  2.10597065e-01  -4.33114237e+05]]


In [44]:
dirCos = batoid.utils.gnomicToDirCos(0.0, 0.01)
rays = batoid.rayGrid(
    fiducial_telescope.dist, fiducial_telescope.pupilSize, 
    dirCos[0], dirCos[1], dirCos[2], 
    nside=16, wavelength=750e-9, medium=fiducial_telescope.inMedium    
)

In [50]:
du = rays.x[1] - rays.x[0]
u1 = np.array([du, 0])
u2 = np.array([0, du])

print(u1)
print(u2)

[ 0.5125  0.    ]
[ 0.      0.5125]


In [52]:
k1 = dkdu[:, 0] * du
k2 = dkdu[:, 1] * du

x1, x2 = batoid.psf.reciprocalLatticeVectors(k1, k2, 16)
print(x1)
print(x2)

print()
x1, x2 = batoid.psf.reciprocalLatticeVectors(k1, k2, 32)
print(x1)
print(x2)

[ -1.70569238e-06   2.60856241e-13]
[  8.29374284e-13  -1.76914552e-06]

[ -8.52846192e-07   1.30428121e-13]
[  4.14687142e-13  -8.84572758e-07]


In [111]:
wavelength = 750e-9

def allLattices(N, du, theta_x, theta_y):
    print("N = {:d}".format(N))
    print("du = {:8.6f}".format(du))
    
    u1 = np.array([du, 0])
    u2 = np.array([0, du])
    print()
    print("u1 = {}".format(u1))
    print("u2 = {}".format(u2))
    
    dkdu = batoid.psf.dkdu(fiducial_telescope, theta_x, theta_y, wavelength, 1024)    
    print()
    print("dkdu = {}".format(dkdu))
    
    k1 = dkdu.dot(u1)
    k2 = dkdu.dot(u2)    
    print()
    print("k1 = {}".format(k1))
    print("k2 = {}".format(k2))    
    
    print("k1/du = {}".format(k1/du))
    print("k2/du = {}".format(k2/du))
    
    x1, x2 = batoid.psf.reciprocalLatticeVectors(k1, k2, N)
    print()
    print("x1 = {}".format(x1))
    print("x2 = {}".format(x2))    
    
    print("x1*du*N = {}".format(x1*du*N))
    print("x2*du*N = {}".format(x2*du*N))
    
    # Converting to tangent plane coords
    dGnomicDFocal = batoid.psf.dthdr(fiducial_telescope, theta_x, theta_y, wavelength)
    print()
    print("dGnomicDFocal = {}".format(dGnomicDFocal))
    gnomic1 = dGnomicDFocal.dot(x1)
    gnomic2 = dGnomicDFocal.dot(x2)
    print("gnomic1 = {}".format(gnomic1))
    print("gnomic2 = {}".format(gnomic2))
    
    print("gnomic1*du*N = {}".format(gnomic1*du*N))
    print("gnomic2*du*N = {}".format(gnomic2*du*N))

    # Converting to spherical coords
    dSphericalDGnomic = batoid.utils.dSphericalDGnomic(theta_x, theta_y)
    print()
    print("dSphericalDGnomic = {}".format(dSphericalDGnomic))
    spherical1 = dSphericalDGnomic.dot(gnomic1)
    spherical2 = dSphericalDGnomic.dot(gnomic2)
    print("spherical1 = {}".format(spherical1))
    print("spherical2 = {}".format(spherical2))

    print("spherical1*du*N = {}".format(spherical1*du*N))
    print("spherical2*du*N = {}".format(spherical2*du*N))

In [112]:
allLattices(16, 0.1, 0.0, 0.01)

N = 16
du = 0.100000

u1 = [ 0.1  0. ]
u2 = [ 0.   0.1]

dkdu = [[ -4.49226436e+05   6.62373549e-02]
 [  2.10597065e-01  -4.33114237e+05]]

k1 = [ -4.49226436e+04   2.10597065e-02]
k2 = [  6.62373549e-03  -4.33114237e+04]
k1/du = [ -4.49226436e+05   2.10597065e-01]
k2/du = [  6.62373549e-02  -4.33114237e+05]

x1 = [ -8.74167347e-06   1.33688824e-12]
x2 = [  4.25054321e-12  -9.06687077e-06]
x1*du*N = [ -1.39866776e-05   2.13902118e-12]
x2*du*N = [  6.80086913e-12  -1.45069932e-05]

dGnomicDFocal = [[  5.36251801e-02  -9.31086192e-07]
 [ -7.25378381e-08   5.16997944e-02]]
gnomic1 = [ -4.68773814e-07   7.03218942e-13]
gnomic2 = [  8.66997433e-12  -4.68755355e-07]
gnomic1*du*N = [ -7.50038102e-07   1.12515031e-12]
gnomic2*du*N = [  1.38719589e-11  -7.50008568e-07]

dSphericalDGnomic = [[ 0.          0.99990001]
 [-0.99995     0.        ]]
spherical1 = [  7.03148627e-13   4.68750377e-07]
spherical2 = [ -4.68708484e-07  -8.66954086e-12]
spherical1*du*N = [  1.12503780e-12   7.50000603e-07]
s

In [113]:
allLattices(32, 0.1, 0.0, 0.01)

N = 32
du = 0.100000

u1 = [ 0.1  0. ]
u2 = [ 0.   0.1]

dkdu = [[ -4.49226436e+05   6.62373549e-02]
 [  2.10597065e-01  -4.33114237e+05]]

k1 = [ -4.49226436e+04   2.10597065e-02]
k2 = [  6.62373549e-03  -4.33114237e+04]
k1/du = [ -4.49226436e+05   2.10597065e-01]
k2/du = [  6.62373549e-02  -4.33114237e+05]

x1 = [ -4.37083674e-06   6.68444118e-13]
x2 = [  2.12527160e-12  -4.53343539e-06]
x1*du*N = [ -1.39866776e-05   2.13902118e-12]
x2*du*N = [  6.80086913e-12  -1.45069932e-05]

dGnomicDFocal = [[  5.36251801e-02  -9.31086192e-07]
 [ -7.25378381e-08   5.16997944e-02]]
gnomic1 = [ -2.34386907e-07   3.51609471e-13]
gnomic2 = [  4.33498716e-12  -2.34377677e-07]
gnomic1*du*N = [ -7.50038102e-07   1.12515031e-12]
gnomic2*du*N = [  1.38719589e-11  -7.50008568e-07]

dSphericalDGnomic = [[ 0.          0.99990001]
 [-0.99995     0.        ]]
spherical1 = [  3.51574314e-13   2.34375189e-07]
spherical2 = [ -2.34354242e-07  -4.33477043e-12]
spherical1*du*N = [  1.12503780e-12   7.50000603e-07]
s

In [114]:
allLattices(32, 0.2, 0.0, 0.01)

N = 32
du = 0.200000

u1 = [ 0.2  0. ]
u2 = [ 0.   0.2]

dkdu = [[ -4.49226436e+05   6.62373549e-02]
 [  2.10597065e-01  -4.33114237e+05]]

k1 = [ -8.98452872e+04   4.21194130e-02]
k2 = [  1.32474710e-02  -8.66228474e+04]
k1/du = [ -4.49226436e+05   2.10597065e-01]
k2/du = [  6.62373549e-02  -4.33114237e+05]

x1 = [ -2.18541837e-06   3.34222059e-13]
x2 = [  1.06263580e-12  -2.26671769e-06]
x1*du*N = [ -1.39866776e-05   2.13902118e-12]
x2*du*N = [  6.80086913e-12  -1.45069932e-05]

dGnomicDFocal = [[  5.36251801e-02  -9.31086192e-07]
 [ -7.25378381e-08   5.16997944e-02]]
gnomic1 = [ -1.17193454e-07   1.75804736e-13]
gnomic2 = [  2.16749358e-12  -1.17188839e-07]
gnomic1*du*N = [ -7.50038102e-07   1.12515031e-12]
gnomic2*du*N = [  1.38719589e-11  -7.50008568e-07]

dSphericalDGnomic = [[ 0.          0.99990001]
 [-0.99995     0.        ]]
spherical1 = [  1.75787157e-13   1.17187594e-07]
spherical2 = [ -1.17177121e-07  -2.16738522e-12]
spherical1*du*N = [  1.12503780e-12   7.50000603e-07]
s